In [ ]:
import hashlib
from importlib import reload
import gc
import math

In [2]:
import shortest_route
from shortest_route.lib import elements
from shortest_route.lib import graph
from shortest_route.lib.graph import CityMapperGraph
from shortest_route.lib.elements import Node, Edge, NodeId, EdgeId, GraphElement
reload(shortest_route)
from shortest_route.lib.graph import CityMapperGraph
from shortest_route.lib.elements import Node, Edge, NodeId, EdgeId, GraphElement

### Load data into nodes and edges

In [3]:
nodes = []
edges = []

with open('../data/shortest_route/citymapper-coding-test-graph.dat', 'r') as f:
    data = f.readlines()
    
# Number of vertices
N = int(data[0].strip("\n"))
for i in range(1, N+1):
    nodes.append(Node(uid_=int(data[i].strip("\n"))))

# Number of edges
E = int(data[N+1])
for i in range(N+2, E+1):
    src, dst, length = data[i].strip("\n").split(" ")
    e_id = hashlib.sha256("{}_{}".format(src,dst).encode()).hexdigest()
    edges.append(Edge(uid_=e_id, src=Node(uid_=int(src)), dst=Node(uid_=int(dst)), length=int(length)))

In [4]:
g = CityMapperGraph()
g.load_nodes_and_edges(nodes=nodes, edges=edges)

In [5]:
src='316319952'
dst=316319241

In [6]:
g.distance(NodeId(uid_=src),NodeId(uid_=dst))

([316319952, 316319241], 36)

In [ ]:
import networkx as nx
G = nx.Graph()
for n in nodes:
    G.add_node(n.uid())
for e in edges:
    G.add_edge(e.src.uid(), e.dst.uid(), weight=e.props.length)
elarge = [(u, v) for (u, v, d) in G.edges(data=True) if d['weight'] == 29]    

In [ ]:
print([p for p in nx.all_shortest_paths(G,source=src,target=dst, weight='weight')])
print(nx.shortest_path_length(G,source=src,target=dst, weight='weight'))

In [ ]:
g.edges()[1].uid()

In [ ]:
Node(uid_=316319241) in g.nodes()

In [7]:
Node.from_uid(NodeId(uid_='316319952')).uid()

316319952